In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# load local version of skaro, before pip installed version, for debugging
import os
import pathlib
import sys

sys.path.append(str(pathlib.Path(os.getcwd()).parent.joinpath("src")))

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yt
from matplotlib.ticker import ScalarFormatter

from skaro.analysis import count_planets
from skaro.model import Model
from skaro.utilities.dataframe import (
    rename_labels,
    rename_entries,
    aggregated_dataframe,
)
from skaro.utilities.math import calculate_pca
from skaro.visualization.visualization_utils import (
    FigureProcessor,
    adjust_legend,
    get_palette,
    ridgeplot,
    set_plot_defaults,
)

## DATA SETUP

In [4]:
snapshot = 127
resolution = 8192
sim_id = "37_11"
num_embryos = 50
host_star_masses = 1
planet_categories = [
    "Earth",
    "Super-Earth",
    "Neptunian",
    "Sub-Giant",
    "Giant",
]

In [5]:
model = Model(
    snapshot=snapshot,
    resolution=resolution,
    sim_id=sim_id,
)
ds = model.ds
mw = model.halo


      DETECTED LOCAL MACHINE: Test snapshot loaded.


      CONFIG: Setting resolution to 4096.


      CONFIG: Setting sim_id to 09_18.



yt : [INFO     ] 2024-03-13 16:08:24,381 Calculating time from 1.000e+00 to be 4.318e+17 seconds
yt : [INFO     ] 2024-03-13 16:08:24,560 Parameters: current_time              = 4.318436660023809e+17 s
yt : [INFO     ] 2024-03-13 16:08:24,560 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-03-13 16:08:24,561 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-03-13 16:08:24,561 Parameters: domain_right_edge         = [100. 100. 100.]
yt : [INFO     ] 2024-03-13 16:08:24,562 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2024-03-13 16:08:24,562 Parameters: current_redshift          = 4.440892098500626e-16
yt : [INFO     ] 2024-03-13 16:08:24,564 Parameters: omega_lambda              = 0.682
yt : [INFO     ] 2024-03-13 16:08:24,564 Parameters: omega_matter              = 0.318
yt : [INFO     ] 2024-03-13 16:08:24,565 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2024-03-13 16:08:24,565 Parameters: hubble_constant   

## PLOT SETUP

In [6]:
set_plot_defaults()

figure_subdirectory = f"MilkyWayAnalogue/{model.resolution}/{model.sim_id}/{model.snapshot}/Component_populations"
save = False

## CREATE DATASOURCE

In [7]:
radius = 0.2 * mw.virial_radius()
masses = (0.3, 0.5, 1)
components = ["Bulge", "Thin Disk", "Thick Disk", "Halo"]

sphere = mw.sphere(radius=radius)

## PLANET AGES PER COMPONENT

In [8]:
# data = aggregated_dataframe(
#     ["bulge_stars", "thin_disk_stars", "thick_disk_stars", "halo_stars"],
#     ["stellar_age", *planet_categories],
#     data_source=sphere,
#     type_name="Component",
#     custom_units={"stellar_age": "Gyr"},
# )
# data = rename_entries(data)

In [9]:
# # average planet age
# data.groupby("Component").apply(
#     lambda x: np.average(
#         x["stellar_age"],
#         weights=sum([x[type] for type in planet_categories]),
#     ),
#     include_groups=False,
# ).round(decimals=1)

In [10]:
# # planet age per type
# planet_ages = pd.DataFrame(
#     [
#         data.groupby("Component").apply(
#             lambda x: np.average(
#                 x["stellar_age"],
#                 weights=x[type],
#             ),
#             include_groups=False,
#         )
#         for type in planet_categories
#     ],
#     index=planet_categories,
# ).T
# planet_ages.round(decimals=1)

## PLANET-HOSTING FRACTION

In [11]:
# fraction_dataframe = count_planets(
#     model,
#     lambda: mw.sphere(radius=radius),
#     planet_categories,
#     model_config={"ngpps_star_masses": 1},
#     long_format=False,
#     fraction=True,
#     value_name="Occurence Rate",
# )
# fraction_dataframe

## OCCURENCE RATES PER COMPONENT

In [12]:
occurence_rate_dataframe = {}
for m in masses:
    occurence_rate_dataframe[m] = count_planets(
        model,
        lambda: mw.sphere(radius=radius),
        planet_categories,
        normalize_by="planet_hosting_number",
        model_config={"ngpps_star_masses": m},
        long_format=False,
        value_name="Occurence Rate",
    )

occurence_rate_long_format = {
    m: occurence_rate_dataframe[m].melt(
        id_vars="Component",
        var_name="Planet Type",
        value_name="Occurence Rate",
    )
    for m in masses
}


      CONFIG: Setting ngpps_star_masses to 0.3.




      TIMER: Block 'Adding Planets...' took 1.7 seconds.


      CONFIG: Setting ngpps_star_masses to 0.5.


      TIMER: Block 'Adding Planets...' took 1.67 seconds.



In [ ]:
# def barplot(occurence_rate_long_format, host_star_mass):
#     plt.figure()
#     with sns.axes_style("ticks"):
#         occurence_rate_barplot = sns.barplot(
#             data=occurence_rate_long_format,
#             x="Planet Type",
#             y="Occurence Rate",
#             hue="Component",
#             hue_order=components,
#             # log=True,
#             palette=get_palette(),
#         )
#     occurence_rate_barplot.set_xlabel("")

#     FigureProcessor(occurence_rate_barplot.figure).save(
#         f"occurence_rates_with_host_star_mass={host_star_mass}.pdf",
#         sub_directory=figure_subdirectory,
#         save=save,
#     )


# for m in masses:
#     plot = barplot(occurence_rate_long_format[m], host_star_mass=m)

In [ ]:
# create table of occurence rates
occurence_rate_combined = {}

for key, value in occurence_rate_dataframe.items():
    dataframe = value.copy()

    # transpose dataframe
    dataframe.set_index("Component", inplace=True)
    dataframe.index.name = None
    dataframe = dataframe.T

    dataframe = dataframe[components]  # reorder columns

    occurence_rate_combined[key] = dataframe

pd.concat(occurence_rate_combined, names=[r"$M_\star$"]).round(decimals=3)

NameError: name 'occurence_rate_dataframe' is not defined

In [ ]:
# switch indices around to compare it on a planet-type basis
pd.concat(occurence_rate_combined, names=[r"$M_\star$"]).swaplevel(
    0, 1
).sort_index().round(decimals=3)

Bulge  Thin Disk  Thick Disk   Halo
            $M_\star$                                     
Earth       0.3        4.901      5.067       4.468  4.621
            0.5        4.740      5.028       4.689  4.859
            1.0        4.080      4.047       5.502  5.236
Giant       0.3        0.000      0.000       0.000  0.000
            0.5        0.016      0.018       0.003  0.004
            1.0        0.286      0.286       0.020  0.062
Neptunian   0.3        0.102      0.109       0.071  0.079
            0.5        0.299      0.357       0.056  0.094
            1.0        0.535      0.562       0.128  0.209
Sub-Giant   0.3        0.013      0.014       0.003  0.004
            0.5        0.059      0.066       0.008  0.010
            1.0        0.120      0.130       0.018  0.039
Super-Earth 0.3        1.997      2.062       1.708  1.759
            0.5        2.824      3.383       1.066  1.428
            1.0        3.779      4.100       2.615  2.988

In [ ]:
print(  # switch indices around to compare it on a planet-type basis
    pd.concat(occurence_rate_combined, names=[r"$M_\star$"])
    .swaplevel(0, 1)
    .sort_index()
    .round(decimals=3)
)